In [1]:
import pandas as pd
import progressbar
import csv
import re
import pickle
import ftfy

In [2]:
df = pd.read_csv(r'ementas_novas.csv').set_index('Unnamed: 0')
df.index.name = None

In [3]:
len(df)

11380

In [4]:
# Removendo linhas duplicadas
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

In [5]:
len(df)

8383

In [6]:
# Tratando o texto
widgets = [' [',
         progressbar.Timer(),
         '] ',
           progressbar.Bar('*'),' (',
           progressbar.ETA(), ') ',
          ]
bar = progressbar.ProgressBar(maxval = len(df), widgets=widgets).start()

for linha in range(len(df)):

    # Removendo possíveis erros de encoding
    df.loc[linha, 'Texto tratado'] = ftfy.fix_text(df['Texto semi-tratado'][linha])

    # Removendo espaços duplos
    df.loc[linha, 'Texto tratado'] = re.sub(r'\s\s+', ' ', df['Texto tratado'][linha]).strip()

    # Corrigindo pontos com espaçamento extra
    df.loc[linha, 'Texto tratado'] = re.sub(r'\s\.\s', '. ', df['Texto tratado'][linha])

    # Substituindo abreviações, seguindo o Código Civil Comentado de Nelson Júnior
    # JÚNIOR, Nelson; NERY, Rosa. Abreviaturas In: JÚNIOR, Nelson; NERY, Rosa. Código Civil Comentado. São Paulo (SP):Editora Revista dos Tribunais. 2014. Disponível em: https://www.jusbrasil.com.br/doutrina/codigo-civil-comentado/1499795013. Acesso em: 8 de Junho de 2024.
    df.loc[linha, 'Texto tratado'] = re.sub(r'câm\.civ\.', 'Câmara Cível', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'câm\.crim\.', 'Câmara Criminal', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'câm\.dir\.emp\.', 'Câmara Especial de Direito Empresarial', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'câmdpriv\.', 'Câmara de Direito Privado', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'câm\.dir\.púb\.', 'Câmara de Direito Público', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'câm\.esp\.', 'Câmara Especial', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'const\.', 'Constituição', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'câm\.', 'Câmara', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'cap\.', 'capítulo', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'art\.', 'artigo', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'ac\.', 'acórdão', df['Texto tratado'][linha], flags=re.IGNORECASE)
    df.loc[linha, 'Texto tratado'] = re.sub(r'cf\.', 'conforme', df['Texto tratado'][linha], flags=re.IGNORECASE)

    bar.update(linha)

In [7]:
# Removendo linhas que possívelmente contem typos ao verificar por caracteres indesejáveis
pattern = re.compile(r'[\^~´`]|ç\s|ç[0-9]|\sb\s|\sf\s|\sg\s|\sj\s|\sq\s|\sr\s|\st\s|\sv\s|\sw\s|\sy\s|\sz\s', re.IGNORECASE)

filter = pd.Series([bool(pattern.findall(df['Texto tratado'][i])) for i in range(len(df))])

In [8]:
df = df[~filter]

In [9]:
df.reset_index(inplace=True, drop=True)

In [10]:
len(df)

7897

In [11]:
df.groupby('Assunto').count()['Texto tratado']

Assunto
Direito Civil             1312
Direito Penal              918
Direito Previdenciário    2076
Direito Tributário        2700
Direito do Trabalho        891
Name: Texto tratado, dtype: int64

In [12]:
# Salvando o objeto dataframe
with open(r'df-novos-pre-processado.pickle', 'wb') as picklefile:
    pickle.dump(df, picklefile, protocol=pickle.HIGHEST_PROTOCOL)